Image generation

In [ ]:

!pip install -q diffusers
!pip install -q accelerate
!pip install -q transformers
!pip install -q bsrgan



import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from bsrgan import BSRGAN
from PIL import Image


base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_8step_unet.safetensors" 


unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda:1", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda:1"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda:1")
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

two_x_model = BSRGAN(model_path='kadirnar/BSRGANx2', device='cuda:0', hf_model=True)


In [ ]:
def predict(prompt):
    img=pipe(prompt, num_inference_steps=8, guidance_scale=0).images[0]
    img.save("1.png")
    pred_two_x = two_x_model.predict(img_path="/kaggle/working/1.png")
    img_2x=Image.open('/kaggle/working/results/BSRGANx2.png')
    return img_2x


In [ ]:
prompt = "A full portrait image of a little girl wearing indian wedding dress "
img=predict(prompt)
print(img.size)

In [ ]:
img